In [9]:
import torch
import numpy as np
import SharedArray as sa
import taichi as ti
import taichi.math as tm

In [4]:
ti.init(arch=ti.cpu, default_fp=ti.f32)

[Taichi] Starting on arch=x64


In [7]:
feat = sa.attach("qopen").reshape(-1, 6000)

In [8]:
feat.shape

(11664, 6000)

In [21]:
@ti.data_oriented
class TsRank:
    def __init__(self, operand: ti.types.f32):
        pass

    @ti.kernel
    def _apply(self, operand: ti.types.ndarray(ndim=2)):
        window = 5
        res = operand.unfold(0, window, 1)
        mask = torch.isnan(res)
        validnum = torch.sum(torch.isfinite(res), dim=1, keepdim=True)
        low = torch.sum(res < res[[0], :], dim=1, keepdim=True)
        high = validnum - torch.sum(res > res[[0], :], dim=1, keepdim=True)
        rank = (high + low - 1) / (validnum - 1) - 1
        rank[validnum == 1] = 0
        rank[mask[0]] = torch.tensor(float("nan"))
        return rank


ts = TsRank(feat)._apply(feat)

TaichiCompilationError: 
File "/tmp/ipykernel_27506/703058511.py", line 8, in _apply:
        res = operand.unfold(0, window, 1)
              ^^^^^^^^^^^^^^
Traceback (most recent call last):
  File "/home/ray/anaconda3/envs/qlib/lib/python3.8/site-packages/taichi/lang/ast/ast_transformer_utils.py", line 27, in __call__
    return method(ctx, node)
  File "/home/ray/anaconda3/envs/qlib/lib/python3.8/site-packages/taichi/lang/ast/ast_transformer.py", line 899, in build_Attribute
    node.ptr = getattr(node.value.ptr, node.attr)
AttributeError: 'AnyArray' object has no attribute 'unfold'


In [33]:
import taichi as ti
import numpy as np


@ti.data_oriented
class TsRank:
    def __init__(self, shape):
        self.rank = ti.field(dtype=ti.f32, shape=shape)

    @ti.kernel
    def _apply(self, operand: ti.types.ndarray(ndim=2)):
        window = 5
        rows, cols = operand.shape[0], operand.shape[1]

        for i in range(rows):
            for j in range(cols):
                if i < window - 1:
                    continue  # Skip the nan assignment

                values = ti.field(dtype=ti.f32, shape=(cols,))
                for k in ti.static(range(cols)):
                    values[k] = operand[i, k]

                current_val = operand[i, j]
                less_than = sum([1 for v in values if v < current_val])
                greater_than = sum([1 for v in values if v > current_val])
                valid_num = sum([1 for v in values if not ti.is_inf(v)])

                if valid_num == 1:
                    self.rank[i, j] = 0
                else:
                    self.rank[i, j] = (less_than + greater_than - 1) / (
                        valid_num - 1
                    ) - 1

        return self.rank


# Assuming feat is a 2D numpy array
feat = np.random.rand(100, 50).astype(np.float32)
test = TsRank(feat.shape)._apply(feat)

TaichiCompilationError: 
File "/tmp/ipykernel_27506/3714816978.py", line 19, in _apply:
                values = ti.field(dtype=ti.f32, shape=(cols,))
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Traceback (most recent call last):
  File "/home/ray/anaconda3/envs/qlib/lib/python3.8/site-packages/taichi/lang/ast/ast_transformer_utils.py", line 27, in __call__
    return method(ctx, node)
  File "/home/ray/anaconda3/envs/qlib/lib/python3.8/site-packages/taichi/lang/ast/ast_transformer.py", line 581, in build_Call
    node.ptr = func(*args, **keywords)
  File "/home/ray/anaconda3/envs/qlib/lib/python3.8/site-packages/taichi/lang/util.py", line 320, in wrapped
    assert in_python_scope(), f"{func.__name__} cannot be called in Taichi-scope"
AssertionError: field cannot be called in Taichi-scope


In [2]:
from alphagen.data.expression_ocean import TsMax